# LSTM

In [ ]:
from src.rnn.architecture import RnnArch
from src.rnn.kfold import RnnFoldCrossValidation
from src.rnn.model import RnnConstructorArgs, RnnModelInitializeArgs, RnnIntMultiRunTrainArgs,\
    RnnIntTrainArgs

## Model Selection

In [ ]:
# Control
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="control-model",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH0,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=5,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# Dnn mimic
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="dnn-mimic-kf",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.DNNMIMIC,
            input_width=1,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=5,
    train_args=RnnIntTrainArgs(
        epochs=10,
        balanced=False,
        augmented=False
    )
))

In [ ]:
# Dnn mimic balanced
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="dnn-mimic-balanced-kf",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.DNNMIMIC,
            input_width=1,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=5,
    train_args=RnnIntTrainArgs(
        epochs=10,
        balanced=True,
        augmented=False
    )
))

In [ ]:
# Dnn mimic balanced and augmented
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="dnn-mimic-balanced-augmented-kf",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.DNNMIMIC,
            input_width=1,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10,
        balanced=True,
        augmented=True
    )
))

In [ ]:
# RNN arch 1, small width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-small-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 1, small width, dense, balanced
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-small-dense-balanced",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10,
        balanced=True
    )
))

In [ ]:
# RNN arch 1, medium width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-medium-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=15,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 1, medium width, dense, balanced
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-medium-dense-balanced",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=15,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10,
        balanced=True
    )
))

In [ ]:
# RNN arch 1, large width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-large-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=30,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 1, large width, dense, balanced
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-large-dense-balanced",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=30,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10,
        balanced=True
    )
))

In [ ]:
# RNN arch 1, small width, sparse
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-small-sparse",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=5,
            spacing=3
        )
    )
)
rnn_folds.train_folds(train_run_args=MultiRunTrainArgs(
    runs=5,
    train_args=TrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 2, small width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch2-small-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH2,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=5,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 2, small width, dense, balanced

rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch2-small-dense-balanced",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH2,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=5,
    train_args=RnnIntTrainArgs(
        epochs=10,
        balanced=True
    )
))

In [ ]:
# RNN arch 3, small width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch3-small-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH3,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=5,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 3, small width, dense, balanced
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch3-small-dense_balanced",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH3,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=5,
    train_args=RnnIntTrainArgs(
        epochs=10,
        balanced=True
    )
))

In [ ]:
# RNN arch 4, small width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch4-small-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH4,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, train_args=RnnIntTrainArgs(epochs=10)
    )
)

In [ ]:
# RNN arch 4, small width, dense, balanced
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch4-small-dense-balanced",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH4,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, 
        train_args=RnnIntTrainArgs(
            epochs=10,
            balanced=True
        )
    )
)

In [ ]:
# RNN arch 5, small width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch5-small-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH5,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, 
        train_args=RnnIntTrainArgs(
            epochs=10
        )
    )
)

In [ ]:
# RNN arch 5, small width, dense, balanced
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch5-small-dense-balanced",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH5,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, 
        train_args=RnnIntTrainArgs(
            epochs=10,
            balanced=True
        )
    )
)

In [ ]:
# RNN arch 6, small width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch6-small-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH6,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, 
        train_args=RnnIntTrainArgs(
            epochs=10
        )
    )
)

In [ ]:
# RNN arch 6, small width, dense, balanced
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch6-small-dense-balanced",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH6,
            input_width=5,
            spacing=1
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, 
        train_args=RnnIntTrainArgs(
            epochs=10,
            balanced=True
        )
    )
)